In [ ]:
!pip install pdfplumber -q
!pip install -U FlagEmbedding -q
!pip install gradio --q
!pip install -q llama-index
!pip install -q llama-core
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-embeddings-huggingface-api
!pip install groq -q
!pip install rank_bm25 -q
!pip install --upgrade huggingface_hub -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.10.1 requires cubinlinker, which is not installed.
cudf 24.10.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.10.1 requires libcudf==24.10.*, which is not installed.
cudf 24.10.1 requires ptxcompiler, which is not installed.
cuml 24.10.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 24.10.0 requires cuvs==24.10.*, which is not installed.
cuml 24.10.0 requires nvidia-cublas, which is not installed.
cuml 24.10.0 requires nvidia-cufft, which is not installed.
cuml 24.10.0 requires nvidia-curand, which is not installed.
cuml 24.10.0 requires nvidia-cusolver, which is not installed.
cuml 24.10.0 requires nvidia-cusparse, which is not installed.
dask-cudf 24.10.1 requires cupy-cuda11x>=12.0.0, which is not installed.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]

In [ ]:
import re
import pdfplumber
import os
import torch
import numpy as np
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, AutoTokenizer, AutoModelForSequenceClassification
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.retrievers import VectorIndexRetriever

from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt_tab')

from groq import Groq
import gradio as gr

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
def extract_text_without_header(pdf_path):
    text = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()

            if page_text:
                page_text = page_text.split("\n", 1)[-1] # remove header

            text += page_text + "\n" if page_text else "\n"

    return text


def split_text(text):
    sections = text.split("§")
    return sections[1:]

In [ ]:
pdf_file_path = "/kaggle/input/history-book/Istoriia-ukrainy-7-klas-Sorochynska-2020.pdf"

In [ ]:
pdf_text = extract_text_without_header(pdf_file_path)

In [ ]:
sections = split_text(pdf_text)
len(sections)

23

In [ ]:
pdf_text[:300]

'Виникнення і становлення Русі-України\nbohdan-books.com/\nПовторимо вивчене з історії України у 6 класі upload/data_files/\ntmp_catalog/\nupovt.pdf\n§ 1. ВСТУП ДО СЕРЕДНЬОВІЧНОЇ ІСТОРІЇ УКРАЇНИ\n1. Історія України як наука і навчальний предмет\nУ 7 класі ви серед інших навчальних предметів продовжите вивче'

In [ ]:
def clean_sections(sections):
    cleaned_sections = []
    for section in sections:
        section = section.replace("bohdan-books.com/", "")
        section = section.replace("upload/data_files/", "")
        section = section.replace("tmp_catalog/", "")
        pattern = r'\b\w+\.pdf\b'

        cleaned_section = re.sub(r'(\w+)\s*-\s*\n\s*(\w+)', r'\1\2', section)
        cleaned_section = re.sub(r'(\S)\s*\n\s*(\S)', r'\1 \2', cleaned_section)
        cleaned_section = re.sub(pattern, '', cleaned_section)
        cleaned_sections.append(cleaned_section)
    return cleaned_sections

In [ ]:
cleaned_sections = clean_sections(sections)

In [ ]:
print(cleaned_sections[3][:500])

 4. КНЯЗЮВАННЯ ІГОРЯ ТА ОЛЬГИ  1. Правління князя Ігоря Наступником Олега на київському престолі став син Рюрика — Ігор. Князь Ігор (? — 945) продовжував справу свого попередника, згуртовуючи східних слов’ян у єдину державу. Розпочав своє правління боротьбою з деревлянами та уличами, які вийшли з покори Києву. Ігор наклав на них значно більшу данину, ніж раніше. У відповідь уличі залишили Середнє Подніпров’я і переселилися в межиріччя Дністра і Південного Бугу. У 915 р. біля кордонів Київської д


In [ ]:
def save_as_txt(content, filename, directory):
    with open(os.path.join(directory, f"{filename}.txt"), 'w', encoding='utf-8') as file:
        file.write(content)

def save_sections_as_files(sections, output_dir, file_format='txt'):
    os.makedirs(output_dir, exist_ok=True)

    for idx, section_content in enumerate(sections, start=1):
        filename = f"Section_{section_content['paragraph_number']}_{section_content['point_number']}"
        if file_format.lower() == 'txt':
            save_as_txt(section_content['text'], filename, output_dir)

    print(f"Sections saved as {file_format.upper()} files in: {output_dir}")

In [ ]:
def split_to_points(sections):
    points_sections = []

    for sec in cleaned_sections:
        idx = sec.find("ВИСНОВКИ")
        sec_no_q = sec[:idx]
        points = re.split(r'(?=\b(?:1[0-9]|2[0-3]|[1-9])[-]?(?:[0-9]+)?\.\s)', sec_no_q)
        title = points[0] + points[1]
        dot_idx = title.find(".")
        title_number = title[1:dot_idx]
        for i, point in enumerate(points[2:], start=1):
          point = title + point
          points_sections.append({"text": point, "paragraph_number": title_number, "point_number": i})

    return points_sections

In [ ]:
points_sections = split_to_points(cleaned_sections)
save_sections_as_files(points_sections, "/kaggle/working/point_sections", file_format='txt')

Sections saved as TXT files in: /kaggle/working/point_sections


# Semantic Search

In [ ]:
def create_index_from_documents(directory_path):
    embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large")
    reader = SimpleDirectoryReader(input_dir=directory_path)
    documents = reader.load_data()

    Settings.embed_model = embed_model
    Settings.llm = None

    index = VectorStoreIndex.from_documents(
      documents,
    )
    return index

In [ ]:
def extract_numbers(filename):
    match = re.match(r'Section_(\d+[-\d]*)(?:[-_](\d+))?.txt', filename)

    if match:
        paragraph_number = match.group(1)  # Paragraph number
        point_number = match.group(2) if match.group(2) else None  # Point number (if present)
        return paragraph_number, point_number
    else:
        print(filename)
        return None, None

In [ ]:
def semantic_search(query, index, top_n=3):
    retriever = index.as_retriever(similarity_top_k=top_n)
    response = retriever.retrieve(query)
    retrieved_documents = []
    for node in response:
        file_name = node.node.metadata["file_name"]
        paragraph_number, point_number = extract_numbers(file_name)

        retrieved_documents.append({"text": node.node.text,
                                    "score": node.score,
                                    "paragraph_number": paragraph_number,
                                    "point_number": point_number
                                   })

    return retrieved_documents

# BM25

In [ ]:
def bm25_search(query, points_sections, top_n=3):
    point_sections_text = [point["text"] for point in points_sections]
    points_sections_lowercase = list(map(str.lower, point_sections_text))
    corpus = [word_tokenize(doc) for doc in points_sections_lowercase]

    bm25 = BM25Okapi(corpus)

    query_tokens = word_tokenize(query.lower())

    scores = bm25.get_scores(query_tokens)

    top_indices = np.argsort(scores)[-top_n:][::-1]

    top_docs = [{"text": points_sections[i]['text'],
                 "score": scores[i],
                 "paragraph_number": points_sections[i]["paragraph_number"],
                "point_number": points_sections[i]["point_number"]
                }
                for i in top_indices]

    return top_docs

# Reranker

In [ ]:
rom FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

def use_reranker(query, documents, top_k=3):
    pairs = [[query, doc["text"]] for doc in documents]
    scores = reranker.compute_score(pairs, normalize=True)
    top_indices = np.argsort(scores)[-top_k:][::-1]
    top_docs = [documents[i] for i in top_indices]
    return top_docsf

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

# LLM

In [ ]:
def create_context(docs):
  context = ''
  for i, doc_info in enumerate(docs, start=1):
      context += f"Документ {i}: " + doc_info['text'] + "\n"
  return context


In [ ]:
directory_path = "/kaggle/working/point_sections"
index = create_index_from_documents(directory_path)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def perform_rag(query, index, points_sections, method='semantic', top_n=3, rerank=False):
    if method == 'bm25':
        print("Performing BM25 Search...")
        top_docs = bm25_search(query, points_sections, top_n)
    elif method == 'semantic':
        print("Performing Semantic Search...")
        top_docs = semantic_search(query, index, top_n)
    else:
        print("No search...")
        return []
    if rerank:
        print("\nResults with reranker:")
        top_docs = use_reranker(query, top_docs, top_k=top_n)
    return top_docs

def perform_llm(query, use_context, groq_api_key, top_docs=None, system_prompt="", temperature=0.7):
    if use_context == 1 and top_docs:
        context = create_context(top_docs)
        llm_query = system_prompt + "Питання: " + query + "Контекст: " + context + " Відповідь: "
    else:
        llm_query = system_prompt + "Питання: " + query +  "Контекст: []" + " Відповідь: "

    client = Groq(api_key=groq_api_key)

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": llm_query,
            }
        ],
        model="gemma-7b-it",
        temperature=temperature
    )

    return chat_completion.choices[0].message.content

# Web interface

In [ ]:
def create_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## Пошук документів і використання LLM для відповіді на питання про книжку з Історії України 7 клас")
        gr.Markdown("Цей сервіс дозволяє виконувати семантичний пошук, BM25 пошук, та реранкінг документів із можливістю використання контексту для запитів до LLM. ")
        gr.Markdown("no-search означає, що для відповіді LLM буде використовувати свої власні знання. Якщо робити пошук, але не давати його в контекст, то LLM не має давати відповіді.")
        PDF_URL = "https://files.pidruchnyk.com.ua/uploads/book/7-klas-istoriia-ukrainy-sorochynska-2020.pdf"
        gr.Markdown(f'<p style="font-size: 18px;">Переглянути книгу можна за посиланням <a href="{PDF_URL}" style="color: orange; text-decoration: underline;">7 клас Історія України - Сорочинська (2020)</a></p>')
        with gr.Row():
            with gr.Column():
                query_input = gr.Textbox(label="Запит", placeholder="Наприклад, Що таке «Руська правда»?")

                method_input = gr.Radio(
                    choices=["semantic", "bm25", "no search"], label="Метод пошуку", value="semantic"
                )

                groq_key_input = gr.Textbox(label="API ключ Groq", type="password")

                rerank_input = gr.Checkbox(label="Використовувати реранкер", value=False)

                temperature_input = gr.Slider(minimum=0, maximum=1, step=0.1, label="Температура", value=0.5)
                use_context = gr.Checkbox(label="Використовувати контекст", value=True)



            with gr.Column():
                results_output = gr.Textbox(label="Результати пошуку та відповіді на питання", interactive=False)

                doc1_output = gr.Textbox(label="Документ 1", interactive=False)
                doc2_output = gr.Textbox(label="Документ 2", interactive=False)
                doc3_output = gr.Textbox(label="Документ 3", interactive=False)

        def choose_prompt(method):
            if method == 'no search':
                return """
                Ви спеціалізований агент, який відповідає на запитання. Відповідай ЛИШЕ українською мовою.
                """
            else:
                return """
                Ви спеціалізований агент, який відповідає на запитання, використовуючи тільки надані документи. Ваше завдання — вибрати найрелевантніші документи із доступних, які відповідають на запитання, і надати відповідь, базуючись виключно на них. Відповідь повинна містити посилання на документ з контексту, з якого була отримана інформація (наприклад, [1]). Якщо контекст порожній або всі документи не відповідають на запитання, ви НЕ надаєте відповідь. Якщо надано контекст, він буде вказаний після запитання в такому форматі: 'Питання: [запитання]. Контекст: [Документ 1. ... Документ 2. ...]'. ЗАВЖДИ використовуйте тільки інформацію з вибраних документів і НЕ використовуйте свої попередні знання, навіть якщо контексту немає. У разі відсутності контексту або якщо жоден документ не відповідає запитуваному, НЕ давайте відповіді. Надавайте відповідь лише українською мовою.
                """



        def process_query(query, method, groq_api_key, rerank, temperature, use_context):
            top_docs = perform_rag(query, index, points_sections, method=method, top_n=3, rerank=rerank)
            system_prompt = choose_prompt(method)
            answer = perform_llm(query, use_context, groq_api_key=groq_api_key, top_docs=top_docs, system_prompt=system_prompt, temperature=temperature)
            docs_texts = [f"Параграф {doc['paragraph_number']}, підпункт {doc['point_number']}. {doc['text']}" for doc in top_docs]
            while len(docs_texts) < 3:
                docs_texts.append("")
            return answer, docs_texts[0], docs_texts[1], docs_texts[2]

        gr.Button("Виконати пошук та отримати відповідь").click(
            process_query,
            inputs=[query_input, method_input, groq_key_input, rerank_input, temperature_input, use_context],
            outputs=[results_output, doc1_output, doc2_output, doc3_output]
        )

    return demo

In [ ]:
interface = create_interface()
interface.launch()

* Running on local URL:  http://127.0.0.1:7863
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://0472b8475a57dee290.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
